In [ ]:
import face_recognition
import os
import cv2
import numpy as np


#To capture a video, you need to create a VideoCapture object (#0-the default one)
video_cap = cv2.VideoCapture(0)

known_face_encodings = []
known_face_names = []
known_dir = 'known2'
i=0
for carpeta in os.listdir(known_dir):
    known_car = known_dir + '/' + carpeta
    known_face_names.append(carpeta)
    for file in os.listdir(known_car):
        img= face_recognition.load_image_file(known_car + '/' + carpeta)
        img_enc = face_recognition.face_encodings(img)[0]
        known_face_encodings[i].append(img_enc)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Capture frame-by-frame (a single frame)
    ret, frame = video_cap.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        best_matches=[]
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match=np.min(face_distances)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index] and best_match<0.5:
                name = known_face_names[best_match_index]

            face_names.append(name)
            best_matches.append(best_match)
            
    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name, best_match in zip(face_locations, face_names, best_matches):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4; right *= 4; bottom *= 4; left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name+ str(np.round(best_match,3)), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('PI2 - Face recognition', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_cap.release()
cv2.destroyAllWindows()